In [169]:
import pandas as pd

df = pd.read_csv("../data/spotify-tracks-audio-features.csv")
# df = df[["acousticness", "energy", "loudness"]]
dataset = df.values
m, n = dataset.shape
x, y = dataset[:,:n-1], dataset[:,n-1]

In [170]:
import numpy as np

column_mins = []
column_maxs = []

for i in range(n-1):
    column_mins.append(min(x[:,i]))
    column_maxs.append(max(x[:,i]))

x_norm = np.zeros((m, n-1))

for i in range(m):
    for j in range(n-1):
        xi = x[i, j]
        x_norm[i, j] = (xi - column_mins[j]) / (column_maxs[j] - column_mins[j])

x = x_norm.copy()

In [171]:
classes = []

for yi in y:
    if yi not in classes:
        classes.append(yi)

classes.sort()

In [172]:
classes_len = len(classes)

y_one_hot = np.zeros((m, classes_len))

for i in range(m):
    yi = int(y[i])
    y_one_hot[i, yi] = 1

y = y_one_hot.copy()

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [173]:
from sklearn.model_selection import train_test_split

train_percentage = 80
test_size = 1 - (train_percentage/100)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)

In [175]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(20, input_shape=(n-1, 1)))
model.add(Dense(len(classes), activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 20)                1760      
_________________________________________________________________
dense_31 (Dense)             (None, 11)                231       
Total params: 1,991
Trainable params: 1,991
Non-trainable params: 0
_________________________________________________________________


In [176]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100


ValueError: in user code:

    d:\Python38\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    d:\Python38\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    d:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    d:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    d:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    d:\Python38\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    d:\Python38\lib\site-packages\keras\engine\training.py:787 train_step
        y_pred = s